其中将 balloon dataset 转化为 COCO 格式的代码如下所示。

In [1]:
import os.path as osp
import mmcv

def convert_balloon_to_coco(ann_file, out_file, image_prefix):
    data_infos = mmcv.load(ann_file)

    annotations = []
    images = []
    obj_count = 0
    for idx, v in enumerate(mmcv.track_iter_progress(data_infos.values())):
        filename = v['filename']
        img_path = osp.join(image_prefix, filename)
        height, width = mmcv.imread(img_path).shape[:2]

        images.append(dict(
            id=idx,
            file_name=filename,
            height=height,
            width=width))

        bboxes = []
        labels = []
        masks = []
        for _, obj in v['regions'].items():
            assert not obj['region_attributes']
            obj = obj['shape_attributes']
            px = obj['all_points_x']
            py = obj['all_points_y']
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            x_min, y_min, x_max, y_max = (
                min(px), min(py), max(px), max(py))


            data_anno = dict(
                image_id=idx,
                id=obj_count,
                category_id=0,
                bbox=[x_min, y_min, x_max - x_min, y_max - y_min],
                area=(x_max - x_min) * (y_max - y_min),
                segmentation=[poly],
                iscrowd=0)
            annotations.append(data_anno)
            obj_count += 1

    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=[{'id':0, 'name': 'balloon'}])
    mmcv.dump(coco_format_json, out_file)

In [12]:
train_ann_file_path = '/openbayes/input/input0/balloon/train/via_region_data.json'
train_out_file_path = '/openbayes/input/output0/balloon/train_via_region_data.json'
train_image_prefix  = '/openbayes/input/input0/balloon/train/'

val_ann_file_path = '/openbayes/input/input0/balloon/val/via_region_data.json'
val_out_file_path = '/openbayes/input/output0/balloon/val_via_region_data.json'
val_image_prefix  = '/openbayes/input/input0/balloon/val/'

In [13]:
convert_balloon_to_coco(train_ann_file_path, train_out_file_path, train_image_prefix)
convert_balloon_to_coco(val_ann_file_path, val_out_file_path, val_image_prefix)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 61/61, 45.5 task/s, elapsed: 1s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 13/13, 38.1 task/s, elapsed: 0s, ETA:     0s


In [31]:
from mmcv import Config

config = Config.fromfile('yolov3_mobilenetv2_mstrain-416_30e_balloon.py')
print(config.pretty_text)  # 把已有的继承进来，再进行修改

checkpoint_config = dict(interval=2, max_keep_ckpts=2)
log_config = dict(interval=50, hooks=[dict(type='TextLoggerHook')])
custom_hooks = [dict(type='NumClassCheckHook')]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'yolov3_mobilenetv2_mstrain-416_300e_coco_20210718_010823-f68a07b3.pth'
resume_from = None
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_method = 'fork'
auto_scale_lr = dict(enable=False, base_batch_size=192)
model = dict(
    type='YOLOV3',
    backbone=dict(
        type='MobileNetV2',
        out_indices=(2, 4, 6),
        act_cfg=dict(type='LeakyReLU', negative_slope=0.1),
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://mmdet/mobilenet_v2')),
    neck=dict(
        type='YOLOV3Neck',
        num_scales=3,
        in_channels=[320, 96, 32],
        out_channels=[96, 96, 96]),
    bbox_head=dict(
        type='YOLOV3Head',
        num_classes=1,
        in_channels=[96, 96, 96],
        out_channels=[96, 96, 

In [ ]:
!mim train mmdet yolov3_mobilenetv2_mstrain-416_30e_balloon.py

/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Training command is /usr/local/bin/python /output/balloon-detection/mmdetection/mmdet/.mim/tools/train.py yolov3_mobilenetv2_mstrain-416_30e_balloon.py --